<H1>Red Hat NLP Spark Project</H1>

This notebook uses Red Hat's OpenShift data logs and processes them using Drain3. 

The first thing this notebook does is webscrape the OpenShift logs and installs dependencies. After, the logs are then cleaned by removing leading tags, dates, timestamps and other unique identifiers. This is done manually because drain does not do this consistently for all logs. Finally the parsed logs are then fed into Drain3 and then put into a dictionary sorted by cluster ID.

The final step that needs to be done is to sift out the pass and fail logs based on their cluster ID and visualize our data.

In [ ]:
!pip3 install drain3
!pip3 install kafka-python
!pip3 install redis

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from google.colab import files
import pandas as pd
import io
import numpy as np
import dateutil
from dateutil import parser
import textblob
import requests
import matplotlib.pyplot as plt
import json

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, confusion_matrix


import drain3
from drain3 import TemplateMiner
import json
import logging
import sys
from drain3.kafka_persistence import KafkaPersistence
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
import subprocess
import time
import spacy

from xgboost import XGBClassifier

#<H3>Web scraping</H3>

Gathers OpenShift using BeautifulSoup and then returns an array containing all of the logs.

In [ ]:
# Web scraping 
# BeautifulSoup for web scraping


#url core needed to pull
website = "https://gcsweb-ci.apps.ci.l2s4.p1.openshiftapps.com"
base = "https://gcsweb-ci.apps.ci.l2s4.p1.openshiftapps.com/gcs/origin-ci-test/logs/canary-release-openshift-origin-installer-e2e-aws-4.5-cnv/"
ending = "build-log.txt"
finished = "finished.json"
url = "https://gcsweb-ci.apps.ci.l2s4.p1.openshiftapps.com/gcs/origin-ci-test/logs/canary-release-openshift-origin-installer-e2e-aws-4.5-cnv/1300557127638585344/build-log.txt"
page = requests.get(base)   
data = page.text
soup = BeautifulSoup(data)
links = []
for link in soup.find_all('a'):
    links.append(link.get('href'))
links = links[1:-1]

final_array = []
labels_link = []
# create array of urls
for x in range(len(links)):
  final_array.append(str(website) + str(links[x]) + str(ending))
  labels_link.append(str(website) + str(links[x]) + str(finished))

# pull all urls logs and store in 2-d array where array_of_logs[x] is a build-log file and 
# array_of_logs[x][y] is an individual log line split by new line
array_of_logs = []
for x in range(len(final_array)):
  page = urlopen(final_array[x])
  html_bytes = page.read()
  array_of_logs.append(str(html_bytes).split('\\n'))
  
# first log
# print(len(lab))


In [ ]:
#url core needed to pull
website2 = "https://gcsweb-ci.apps.ci.l2s4.p1.openshiftapps.com"
base2 = "https://gcsweb-ci.apps.ci.l2s4.p1.openshiftapps.com/gcs/origin-ci-test/logs/release-openshift-ocp-installer-e2e-aws-serial-4.1/"
ending2 = "build-log.txt"
finished2 = "finished.json"
page2 = requests.get(base2)    
data2 = page2.text
soup2 = BeautifulSoup(data2)
links2 = []
for link2 in soup2.find_all('a'):
    links2.append(link2.get('href'))
links2 = links2[1:-1]

final_array2 = []
labels_link2 = []
# create array of urls
for x in range(len(links2)):
  final_array2.append(str(website2) + str(links2[x]) + str(ending2))
  labels_link2.append(str(website2) + str(links2[x]) + str(finished2))

# pull all urls logs and store in 2-d array where array_of_logs[x] is a build-log file and 
# array_of_logs[x][y] is an individual log line split by new line
array_of_logs2 = []
for x in range(len(final_array2)):
  page2 = urlopen(final_array2[x])
  html_bytes2 = page2.read()
  array_of_logs2.append(str(html_bytes2).split('\\n'))
  
# first log
# print(array_of_logs2[0])

# IMPORTANT

Run this cell below a few time in a row in order for it to completely remove all the necessary things it is supposed to remove.

In [ ]:
for i in range(len(array_of_logs2)):
  #  removing newline characters
  array_of_logs2[i] = str(array_of_logs2[i]).replace('\\n', ' ')
  
 # removes leading 'b from log
  array_of_logs2[i] = array_of_logs2[i][2:]

This code cell above removes newline characters and leading byte signature

<H4>Assigning Labels to Logs </H4>

In [ ]:
labels= []
index = 0
to_remove = []

for x in range(len(labels_link)):
    page = urlopen(labels_link[x])
    data = json.load(page) 
    labels.append(data["result"])
    index += 1

for x in range(len(labels_link2)):
    page = urlopen(labels_link2[x])
    try:
      data = json.load(page)
      labels.append(data["result"])
    except:
      to_remove.append(index)
      continue
    index += 1

array_of_logs2 = array_of_logs + array_of_logs2

for i in range(len(array_of_logs2)):
    if i in to_remove:
      array_of_logs2[i] = None
array_of_logs2 = [log for log in array_of_logs2 if log != None]
print(labels)

['FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', 'SUCCESS'

<H4>Splitting log lines for vectorization</H4>

In [ ]:
vocab = {}
i = 0
for log in array_of_logs2:
  log = str(log).split(",")
  for line in log:
    if line not in vocab:
      vocab[line] = i
      i+=1
print(len(vocab))

40194


In [ ]:
# 1 := success, 0 := fail
y = []
count_0 = 0
count_1 = 0
for label in labels:
  if label == 'SUCCESS':
    y.append(1)
    count_1 = count_1 + 1
  else:
    y.append(0)
    count_0 = count_0 + 1
print("y = ", y)
print("number of success = ", count_1)
print("number of fail = ", count_0)

y =  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
number of success =  140
number of fail =  115


#<H2>Classifying logs without drain and parsing</H2>

Here begins the process of vectorizing each log line and then classifying it as a pass or fail log accordingly. If this cell has an error, run the cell that removes newline characters and the leading b' tag from the logs.

In [ ]:
total = []
for log in array_of_logs2:
  s = ", ".join(log)
  total.append(s)
#print(total)

In [ ]:

vectorizer = TfidfVectorizer(vocabulary = vocab)
X = vectorizer.fit_transform(total)
#X = vectorizer.fit_transform(array_of_logs2)

In [ ]:
X

<255x40194 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [ ]:

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, train_size= 0.8, random_state=1)
sss.get_n_splits(X, y)
y = np.array(y)
for train_index, test_index in sss.split(X, y):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]

Manual splitting to see how classifier is working

In [ ]:
X_train = X[:100]
y_train = y[:100]
X_test = X[100:]
y_test = y[100:]

#<H4>Classifying logs without Drain with Random Forest Classifier

In [ ]:

model_RF = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2,
                               min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, 
                               max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, 
                               bootstrap=True, oob_score=False, n_jobs=None, random_state=1, verbose=0, 
                               warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None).fit(X_train, y_train)

                              

y_test_predictions = model_RF.predict(X_test)
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_test_predictions))

print("=== Classification Report ===")
print(classification_report(y_test, y_test_predictions))


print("y_test_predictions", y_test_predictions)

#print("RMSE on testing set = ", mean_squared_error(y_test, y_test_predictions))

#print("Accuracy on testing set = ",accuracy_score(y_test, y_test_predictions))

=== Confusion Matrix ===
[[ 25   0]
 [130   0]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.16      1.00      0.28        25
           1       0.00      0.00      0.00       130

    accuracy                           0.16       155
   macro avg       0.08      0.50      0.14       155
weighted avg       0.03      0.16      0.04       155

y_test_predictions [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:

print("Precision score: {}".format(precision_score(y_test, y_test_predictions)))
print("Recall score: {}".format(recall_score(y_test, y_test_predictions)))
print("F1 Score: {}".format(f1_score(y_test, y_test_predictions)))

Precision score: 0.0
Recall score: 0.0
F1 Score: 0.0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Classifying logs with Drain with XGB

In [ ]:
model_XGB = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1).fit(X_train, y_train)
y_test_predictions = model_XGB.predict(X_test)

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_test_predictions))

print("=== Classification Report ===")
print(classification_report(y_test, y_test_predictions))

print("y_test_predictions", y_test_predictions)

=== Confusion Matrix ===
[[ 0 23]
 [ 0 28]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        23
           1       0.55      1.00      0.71        28

    accuracy                           0.55        51
   macro avg       0.27      0.50      0.35        51
weighted avg       0.30      0.55      0.39        51

y_test_predictions [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


=== Confusion Matrix ===
[[ 0 23]
 [ 0 28]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        23
           1       0.55      1.00      0.71        28

    accuracy                           0.55        51
   macro avg       0.27      0.50      0.35        51
weighted avg       0.30      0.55      0.39        51

y_test_predictions [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print("Precision score: {}".format(precision_score(y_test, y_test_predictions)))
print("Recall score: {}".format(recall_score(y_test, y_test_predictions)))
print("F1 Score: {}".format(f1_score(y_test, y_test_predictions)))

Precision score: 0.5490196078431373
Recall score: 1.0
F1 Score: 0.7088607594936709


#<H3>Log Parsing</H3>

Logs need to be parsed before processing into drain. This is necessary because drain does not parse consistently for every log.

Things to be parsed:
<ul>
  <li>Dates</li>
  <li>Timestamps</li>
  <li>Newline characters</li>
  <li>Version numbers</li>
  <li>Namespace IDs</li>
  <li>URLs</li>
</ul>

In [ ]:
original_log = array_of_logs2[1]

In [ ]:
#  helper function detecting if a string is a date / timestamp
def is_date(str):
  try:
    dateutil.parser.parse(str)
    return True
  except:
    return False
for i in range(len(array_of_logs2)):
 # splitting each section as its own index (for parsing)
  array_of_logs2[i] = str(array_of_logs2[i]).split(' ')
  for j in range(len(array_of_logs2[i])):
    if is_date(array_of_logs2[i][j]) == True:
      array_of_logs2[i][j] = ''
  array_of_logs2[i] = list(filter(lambda x: x != '', array_of_logs2[i]))
  array_of_logs2[i] = ' '.join(array_of_logs2[i])

This code cell removes timestamps and dates in order to mitigate unique identifiers

In [ ]:
for index in range(len(array_of_logs2)):
  # removing version number
  if "version" in array_of_logs2[index]:
    tmp = array_of_logs2[index].split("version",1)
    tmp_1 = tmp[0]
    tmp_2 = tmp[1].split()
    tmp_2 = tmp_2[1:]
    tmp = ''.join(tmp_1) + ' '.join(tmp_2)
    array_of_logs2[index] = tmp

  # removing creating namespace ID
  if "Creating namespace" in array_of_logs2[index]:
    tmp = array_of_logs2[index].split("Creating namespace", 1)
    tmp_1 = tmp[0]
    tmp_2 = tmp[1].split()
    tmp_2 = tmp_2[1:]
    tmp = ''.join(tmp_1) + ' '.join(tmp_2)
    array_of_logs2[index] = tmp

  # removing using namespace ID
  if "Using namespace" in array_of_logs2[index]:
    tmp = array_of_logs2[index].split("Using namespace", 1)
    tmp_1 = tmp[0]
    tmp_2 = tmp[1].split()
    tmp_2 = tmp_2[1:]
    tmp = ''.join(tmp_1) + ' '.join(tmp_2)
    array_of_logs2[index] = tmp

  # removing Imported release stamp
  if "Imported release" in array_of_logs2[index]:
    tmp = array_of_logs2[index].split("Imported release", 1)
    tmp_1 = tmp[0]
    tmp_2 = tmp[1].split()
    tmp_2 = tmp_2[1:]
    tmp = ''.join(tmp_1) + ' '.join(tmp_2)
    array_of_logs2[index] = tmp

  # removing Acquired lease stamp
  if "Acquired lease" in array_of_logs2[index]:
    tmp = array_of_logs2[index].split("Acquired lease", 1)
    tmp_1 = tmp[0]
    tmp_2 = tmp[1].split()
    tmp_2 = tmp_2[1:]
    tmp = ''.join(tmp_1) + ' '.join(tmp_2)
    array_of_logs2[index] = tmp

  # removing "images will be pullable from" link
  if "images will be pullable from" in array_of_logs2[index]:
    tmp = array_of_logs2[index].split("images will be pullable from", 1)
    tmp_1 = tmp[0]
    tmp_2 = tmp[1].split()
    tmp_2 = tmp_2[1:]
    tmp = ''.join(tmp_1) + ' '.join(tmp_2)
    array_of_logs2[index] = tmp

This code cell is removing any unique identifiers such as version numbers, URLs and namespace IDs

#<H3>Drain3 Processing</H3>

The parsed logs are now being processed in Drain. Drain will do additional parsing and also cluster the logs. Drain uses longest common subsequence as their algorithm for clustering, so logs with similar structure will be considered to be in the same clustering. There are two dictionaries that represent the size of each cluster and another that separates the logs by cluster ID. The goal of this is to determine which clusters are pass logs and which clusters are fail logs.

In [ ]:

template_miner = TemplateMiner(None)
i = 0
ints_from_drain = []
while True:
    if i >= len(array_of_logs2):
      break
    log_line = ' '.join(array_of_logs2[i])
    i += 1
    if log_line == 'q':
        break
    result = template_miner.add_log_message(log_line)
    result_json = json.dumps(result)
    ints_from_drain.append(re.findall(r'\d+', result_json))
    # print(result_json)

ints_from_drain = np.asarray(ints_from_drain)

cluster_size = {}
cluster_id = {}
for cluster in template_miner.drain.clusters:
  if cluster.cluster_id not in cluster_size:
    cluster_size[cluster.cluster_id] = cluster.size
    cluster_id[cluster.cluster_id] = []


for i in range(len(ints_from_drain)):
  cluster_id[int(ints_from_drain[i][0])].append(array_of_logs2[i])

print(cluster_size)


Starting Drain3 template miner
Loading configuration from drain3.ini
config file not found: drain3.ini
{1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 2, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 36: 1, 37: 1, 38: 1, 39: 1, 40: 1, 41: 1, 42: 1, 43: 1, 44: 1, 45: 1, 46: 1, 47: 1, 48: 1, 49: 1, 50: 1, 51: 1, 52: 1, 53: 1, 54: 1, 55: 1, 56: 1, 57: 1, 58: 1, 59: 1, 60: 1, 61: 1, 62: 1, 63: 1, 64: 1, 65: 1, 66: 1, 67: 1, 68: 1, 69: 1, 70: 1, 71: 1, 72: 1, 73: 1, 74: 1, 75: 1, 76: 1, 77: 1, 78: 1, 79: 1, 80: 1, 81: 1, 82: 1, 83: 1, 84: 1, 85: 1, 86: 1, 87: 1, 88: 1, 89: 1, 90: 6, 91: 2, 92: 8, 93: 5, 94: 1, 95: 5, 96: 1, 97: 2, 98: 1, 99: 1, 100: 6, 101: 1, 102: 1, 103: 1, 104: 3, 105: 1, 106: 1, 107: 1, 108: 1, 109: 17, 110: 1, 111: 1, 112: 1, 113: 1, 114: 3, 115: 1, 116: 1, 117: 1, 118: 10, 119: 1, 120: 1, 121: 1, 122: 1, 123: 1, 124: 3, 125: 1, 12

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


#<H3>Parsing Results Print out</H3>

Here is a snippet example of what the parsing process does to each log. As you can see the leading byte tag, dates, and unique URLs will be parsed out from both manual parsing and Drain.

In [ ]:
# print(original_log.split(',')[:3])
print(array_of_logs2[1].split(',')[:3])

['["b\'2020/10/26 ci-operator No source defined\'', " Resolved release latest to registry.svc.ci.openshift.org/ocp/release:4.5-ci'", ' Running [release-inputs]']


<H3>Number of unique logs</H3>
Passed 266 logs into Drain, it returned 155 unique clusters after that.

#<H4>Log cluster example</H4>

Here is a printout of logs that fall under a cluster with the ID of 94. This cluster contains the most amount of data logs. Based on the clustering algorithm it is evident that drain uses longest common sequence because all of these logs in this cluster contain the exact same structure with very small differences, especially after being parsed.

In [ ]:
i = 1
for log in cluster_id[95]:
  print("log", i, ":",  log[:200], "...")
  print()
  i += 1

log 1 : b'2020/11/10 ci-operator No source defined', Resolved release latest to registry.svc.ci.openshift.org/ocp/release:4.1', warning: overriding parameter "LEASED_RESOURCE"', Running [release-inputs], e2e- ...

log 2 : b'2020/11/11 ci-operator No source defined', Resolved release latest to registry.svc.ci.openshift.org/ocp/release:4.1', warning: overriding parameter "LEASED_RESOURCE"', Running [release-inputs], e2e- ...

log 3 : b'2020/11/12 ci-operator No source defined', Resolved release latest to registry.svc.ci.openshift.org/ocp/release:4.1', warning: overriding parameter "LEASED_RESOURCE"', Running [release-inputs], e2e- ...

log 4 : b'2020/11/13 ci-operator No source defined', Resolved release latest to registry.svc.ci.openshift.org/ocp/release:4.1', warning: overriding parameter "LEASED_RESOURCE"', Running [release-inputs], e2e- ...

log 5 : b'2020/11/15 ci-operator No source defined', Resolved release latest to registry.svc.ci.openshift.org/ocp/release:4.1', warning: overrid

#<H2>Classifying logs with Drain and parsing</H2>

In [ ]:
vocab = {}
i = 0
hhh = []
for log in array_of_logs2:
  log = str(log).split(",")
  for line in log:
    if line not in vocab:
      vocab[line] = i
      i+=1
      hhh.append(line)
print(len(vocab))
# print(hhh)

27449


<H4>Vectorizing log lines</H4>

In [ ]:

vectorizer = TfidfVectorizer(vocabulary = vocab)
X = vectorizer.fit_transform(hhh)


In [ ]:

vectorizer = TfidfVectorizer(vocabulary = vocab)
X = vectorizer.fit_transform(total)

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, train_size= 0.8, random_state=1)
sss.get_n_splits(X, y)
y = np.array(y)
for train_index, test_index in sss.split(X, y):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]



#<H4>Classifying logs with Drain with Random Forest Classifier</H4>

In [ ]:

model_RF_D = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2,
                               min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, 
                               max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, 
                               bootstrap=True, oob_score=False, n_jobs=None, random_state=1, verbose=0, 
                               warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None).fit(X_train, y_train)
                              
y_test_predictions = model_XGB.predict(X_test)

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_test_predictions))

print("=== Classification Report ===")
print(classification_report(y_test, y_test_predictions))

print("y_test_predictions", y_test_predictions)
#print("RMSE on testing set = ", mean_squared_error(y_test, y_test_predictions))


#print("Accuracy on testing set = ",accuracy_score(y_test, y_test_predictions))


=== Confusion Matrix ===
[[ 0 23]
 [ 0 28]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        23
           1       0.55      1.00      0.71        28

    accuracy                           0.55        51
   macro avg       0.27      0.50      0.35        51
weighted avg       0.30      0.55      0.39        51

y_test_predictions [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:

print("Precision score: {}".format(precision_score(y_test, y_test_predictions)))
print("Recall score: {}".format(recall_score(y_test, y_test_predictions)))
print("F1 Score: {}".format(f1_score(y_test, y_test_predictions)))

Precision score: 0.5490196078431373
Recall score: 1.0
F1 Score: 0.7088607594936709


# Classifying logs with Drain with XGB

In [ ]:
model_XGB_D = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1).fit(X_train, y_train)
y_test_predictions = model_XGB_D.predict(X_test)
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_test_predictions))

print("=== Classification Report ===")
print(classification_report(y_test, y_test_predictions))
print("y_test_predictions", y_test_predictions)

print("Precision score: {}".format(precision_score(y_test, y_test_predictions)))
print("Recall score: {}".format(recall_score(y_test, y_test_predictions)))
print("F1 Score: {}".format(f1_score(y_test, y_test_predictions)))

=== Confusion Matrix ===
[[ 0 23]
 [ 0 28]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        23
           1       0.55      1.00      0.71        28

    accuracy                           0.55        51
   macro avg       0.27      0.50      0.35        51
weighted avg       0.30      0.55      0.39        51

y_test_predictions [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Precision score: 0.5490196078431373
Recall score: 1.0
F1 Score: 0.7088607594936709


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Gnerate clusters and a prefix tree
The cell below runs the drain3 program. Process the log info and generate clusters and a prefix tree of the log information.

In [ ]:


logger = logging.getLogger(__name__)
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(message)s')

template_miner = TemplateMiner()

line_count = 0
start_time = time.time()
batch_start_time = start_time
batch_size = 10000
for i in range(len(array_of_logs2)):
  for line in array_of_logs2[i]:
      line = line.rstrip()
      line = line.partition(": ")[2]
      result = template_miner.add_log_message(line)
      line_count += 1
      if line_count % batch_size == 0:
          time_took = time.time() - batch_start_time
          rate = batch_size / time_took
          # logger.info(f"Processing line: {line_count}, rate {rate:.1f} lines/sec, "
          #             f"{len(template_miner.drain.clusters)} clusters so far.")
          batch_start_time = time.time()
      if result["change_type"] != "none":
          result_json = json.dumps(result)
          logger.info(f"Input ({line_count}): " + line)
          logger.info("Result: " + result_json)

time_took = time.time() - start_time
rate = line_count / time_took
logger.info(f"--- Done processing file. Total of {line_count} lines, rate {rate:.1f} lines/sec, "
            f"{len(template_miner.drain.clusters)} clusters")
sorted_clusters = sorted(template_miner.drain.clusters, key=lambda it: it.size, reverse=True)
for cluster in sorted_clusters:
    logger.info(cluster)

print("Prefix Tree:")
template_miner.drain.print_tree()

template_miner.profiler.report(0)

Starting Drain3 template miner
Loading configuration from drain3.ini
config file not found: drain3.ini
Input (1): 
Result: {"change_type": "cluster_created", "cluster_id": 1, "cluster_size": 1, "template_mined": "", "cluster_count": 1}
--- Done processing file. Total of 8593683 lines, rate 141959.9 lines/sec, 1 clusters
ID=1     : size=8593683   : 
Prefix Tree:
<root>
	<0>
